In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

# Prepare data for analysis

- Prepared categorical markers

In [ ]:
ii <- readRDS(paste0(SHARE_DIR, "biomarkers_ready.Rds"))
ready <- 
ii$ready %>% 
 rw() %>% mu(groupedTreatmentType = paste0(unique(strsplit(derived_treatmentType, " ## ")[[1]]), collapse = " ## ")) %>% ug()
categorical_features <- ii$features

- Cohorts 

In [ ]:
cohorts <- fread("/mnt/bioinfnas2/immunocomp/shared_reference_data/cohorts/cohorts_ready.csv")

- Make treatments based on cohort and type

In [ ]:
go_treat <- 
rbind(ready %>% lj(cohorts %>% se(sampleId, cohort), by = "sampleId"), ready %>% mu(cohort = "Pan-Cancer")) %>% 
 mu(cohortGo = paste0(cohort, " ## ", derived_treatmentMechanism), group = "mechanism")

In [ ]:
go_type <- 
rbind(ready %>% lj(cohorts %>% se(sampleId, cohort), by = "sampleId"), ready %>% mu(cohort = "Pan-Cancer")) %>% 
 mu(cohortGo = paste0(cohort, " ## ", groupedTreatmentType), group = "type" )

In [ ]:
all <- 
rbind(ready %>% lj(cohorts %>% se(sampleId, cohort), by = "sampleId"), ready %>% mu(cohort = "Pan-Cancer")) %>% 
 mu(cohortGo = "Pan-Cancer", group = "type" )

In [ ]:
go <- go_treat %>% bind_rows(go_type) %>% bind_rows(all)

In [ ]:
fwrite(go, paste0(SHARE_DIR, "fisher_base.csv"))

- Compute and save top mechanisms

In [ ]:
top_mechanisms <- 
go %>% 
 gb(cohortGo, group) %>% 
 su(ct = n(), no_dcb = sum(non_response), dcb = ct - no_dcb) %>% 
 fi(ct > min_patients, no_dcb > min_response, dcb > min_response) %>% 
 ug()

In [ ]:
fwrite(top_mechanisms, paste0(SHARE_DIR, "top_mechanisms.csv"))

In [ ]:
paste0(SHARE_DIR, "top_mechanisms.csv")

# Run RA fisher on prepared counts data

- Run fisher's exact test across all categorical features

In [ ]:
ra_ready <- 
go %>% 
 fi(cohortGo %in% (top_mechanisms %>% pu(cohortGo))) %>% 
 se(cohortGo, non_response, any_of(categorical_features))

In [ ]:
ra_go <- ra_formatter_and_test(ra_ready)

# Send it

In [ ]:
fwrite(ra_go, paste0(SHARE_DIR, "1_run_fishers_exact.csv"))